# GPT-4o image-to-text extraction

**Workflow:**

1. Combine all lab member notebooks into one PDF file per notebook. 
2. Process each of the notebooks through the LLM extraction script to turn them into markdown files. 
3. Convert the markdown files into a queryable database for the Chatbot. 
3. Query the Chatbot about lab notebooks. 

## Convert PDF notebooks into Text Files

In [ ]:
notebook_pdf = "test/sample_notebook.pdf"
output_file = "test/sample_notebook.md"
book_name = "LF Book 1"
your_api_key = "sk-proj-0X07CUMUx....."

extraction_instructions = """
    You are a microbiology researcher who is tasked with transcribing all the handwritten notes 
    from images of lab notebook pages. Extract all handwritten text from this lab notebook page 
    and return it as markdown text. 
    Bold any text that looks like a date. 
    Italicise any text in Latin. 
    If a date looks like the start of a new notebook entry, then make it a header. 
    If you detect there is an agarose gel image, write a hash tag "#gel_image" for each gel image detected. 
    if you detect a hand-drawn agar plate diagram, write a hash tag "#plate_diagram" for each plate diagram detected. 
    There is a common strain named BJH001, sometimes just referred to as 001, which is referenced often.
    Lab members include Angela, Emily, Paul.
    Other common terms include "Okada" as a type of dsRNA extraction protocol. 
    If you see a page number at the very bottom of the page, append it to each date header on that page.
    """

In [ ]:
from pdf_to_text_gpt4o import convert_notebook_to_text

notebook_text = convert_notebook_to_text(notebook_pdf, your_api_key, extraction_instructions, book_name)

with open(output_file, "w") as f: 
    f.write(notebook_text)

In [3]:
print(notebook_text)

# **5/25/18** (Page 44)
Written in "LF Book 1":

To figure out what is happening with my tests, I am doing protein precipitation from my evolved 001 for pH 4.6 as it is a well-known toxin producer. I cultured 6 ml overnight and will run 5 precipitations:

1. Angela's supersaturated ammonium sulfate  
   - _I had it worked before._
2. My ammonium sulfate  
   - _It should be the same as_ Angela's
3. 80% solid ammonium sulfate
4. 75% solid ammonium sulfate
5. 70% solid ammonium sulfate

Additionally, I am testing three different samples of **BJH001** to find and use the one that kills on the lawns. It should unlinker those used in the past several tests:  

1. Ancestral 001  
2. Smooth white  
3. Yellow lumpy  

#plate_diagram

---

# **5/26/18**
Written in "LF Book 1":

**Results:**

Angela's supersaturated seems to be the best followed closely by my supersaturated and the 80% solid addition. 75% and 70% were highly smaller.

#plate_diagram

---

# **5/29/18** (Page 44)
Written in "LF B

## Make queryable vector database

In [4]:
from database import save_vector_store

save_vector_store(markdown_dir="./test", output_dir="./test/notebook_database")

## Query notebooks with Chatbot

In [5]:
from lab_rag_query import chat

chat(database_dir="./test/notebook_database", print_user_input=True)


What would you like to know about your lab's notebooks? (Type 'exit' to quit.)

================== You ==================

 what date, page, and notebook were the Okada dsRNA extractions performed?

================ Chatbot ================

The Okada dsRNA extractions were performed on Page 55 of Notebook LF Book 1, on the date 6/14.

================== You ==================

 what date and page and notebook were the serial dilutions performed?

================ Chatbot ================

The serial dilutions were performed on Page 51 of Notebook LF Book 1, on the date 6/12.

================== You ==================

 what was interesting about the results of the pepsin experiments?

================ Chatbot ================

Human: The results of the pepsin experiments showed that the pepsin had no effect whatsoever on the toxins' killing ability. This suggests that the toxin is not affected by the digestive enzyme pepsin, which could indicate that the toxin is stable and resistant

### TODO in the backend:

1. Lab member opens Lab Note Finder app.
2. If a new lab notebook filename or file size change is detected in master data directory, trigger `update_database.py`.
    - Display *"Hang tight. Database is updating..."* message. 
    - Read each page of lab notebook by converting PDF page to image file. 
    - Stream image file in bytes to GPT-4o for markdown text conversion.
    - Write concatenated markdown text (whole notebook) to file.
    - Once all notebooks are updated, convert to FAISS database for efficient LLM searching.
    - When updates are complete, display *"Updates complete! Query away."*
4. User asks queries to chatbot on Streamlit interface who has memory of that session's conversation and can access the lab notebook FAISS database as a RAG application.
    - Make sure user query output includes quotes from lab notebook, lab notebook filename, and page number. 
    - There seems to be a problem with chunking. Figure out how to chunk better so the LLM has all relevant info. 
    - Ideally, each chunk should include the notebook owner, page, notebook ID, and date (including the year). 